INSTALL
- FAISS - 
- Sent2Vec

In [1]:
import numpy as np
import faiss   

In [2]:
import os
import time
name = "TED-"+ str(time.strftime("%Y%m%d-%H%M"))

!mkdir $name
path = os.getcwd() + "/" + name
os.chdir(path)

!pwd


os.chdir(path)

/home/chinh/clc_data/Script/TED-20210225-1705


In [3]:
!wget https://github.com/hoangtrungchinh/clc_data/raw/master/en_vi_iwslt.tar.gz
!mkdir data_bin
!tar -xvf  'en_vi_iwslt.tar.gz' -C 'data_bin'

--2021-02-25 17:05:07--  https://github.com/hoangtrungchinh/clc_data/raw/master/en_vi_iwslt.tar.gz
Resolving github.com (github.com)... 52.74.223.119
Connecting to github.com (github.com)|52.74.223.119|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/en_vi_iwslt.tar.gz [following]
--2021-02-25 17:05:07--  https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/en_vi_iwslt.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9044100 (8,6M) [application/octet-stream]
Saving to: ‘en_vi_iwslt.tar.gz’

en_vi_iwslt.tar.gz  100%[===================>]   8,62M  1,10MB/s    in 8,5s    

2021-02-25 17:05:16 (1,02 MB/s) - ‘en_vi_iwslt.tar.gz’ saved [9044100/904410

In [4]:
# import file
import numpy as np
import glob

en_files = sorted(glob.glob("data_bin/*.en"))
vi_files = sorted(glob.glob("data_bin/*.vi"))
print(en_files)
print(vi_files)

lst_en = []
lst_vi = []

start = time.time()

for file in en_files:
  with open(file) as file_in:
    for line in file_in:
      lst_en.append(line)

for file in vi_files:
  with open(file) as file_in:
    for line in file_in:
      lst_vi.append(line)

print(len(lst_en))
print(len(lst_vi))

['data_bin/IWSLT15.TED.tst2012.en-vi.en', 'data_bin/IWSLT15.TED.tst2013.en-vi.en', 'data_bin/IWSLT15.TED.tst2015.en-vi.en', 'data_bin/train.en']
['data_bin/IWSLT15.TED.tst2012.en-vi.vi', 'data_bin/IWSLT15.TED.tst2013.en-vi.vi', 'data_bin/IWSLT15.TED.tst2015.en-vi.vi', 'data_bin/train.vi']
120956
120956


In [5]:
# SPLIT TRAIN, TEST, VALID
from sklearn.model_selection import train_test_split
en_train, en_test_valid, vi_train, vi_test_valid =  train_test_split(lst_en, lst_vi, test_size=0.2, random_state=123)
en_valid, en_test, vi_valid, vi_test = train_test_split(en_test_valid, vi_test_valid, test_size=0.5, random_state=123)
print(len(en_train))
print(len(vi_train))
print(len(en_valid))
print(len(vi_valid))
print(len(en_test))
print(len(vi_test))
print(en_train[20])
print(vi_train[20])
print(en_valid[-1])
print(vi_valid[-1])
print(en_test[-1])
print(vi_test[-1])

96764
96764
12096
12096
12096
12096
And , in fact , when we did some experiments looking at this , what we found is that 46 percent of the variation in how many friends you have is explained by your genes .

Trong thực tế , khi thực hiện một vài thí nghiệm nghiên cứu vấn đề này , chúng tôi nhận ra 46 % sự khác biệt trong số bạn bè của bạn được giải thích bằng gen của bạn .

This is a chart of what it looked like when it first became popular last summer .

Đây là biểu đồ thể hiện điều đó khi video này lần đầu tiên trở nên phổ biến vào mùa hè trước .

And what will happen in the future ?

Chuyện gì sẽ xảy ra trong tương lai ?



In [6]:
file_variable = [en_train,vi_train,en_valid,vi_valid,en_test,vi_test]
file_names = ["en_train","vi_train","en_valid","vi_valid","en_test","vi_test"]

for i in range(len(file_names)):
  with open(file_names[i], 'w') as f:
    for item in file_variable[i]:
      f.write(item)
!ls -al

total 36844
drwxrwxr-x 3 chinh chinh     4096 Thg 2  25 17:05 .
drwxrwxr-x 4 chinh chinh     4096 Thg 2  25 17:05 ..
drwxrwxr-x 2 chinh chinh     4096 Thg 2  25 17:05 data_bin
-rw-rw-r-- 1 chinh chinh  1260181 Thg 2  25 17:05 en_test
-rw-rw-r-- 1 chinh chinh 10011509 Thg 2  25 17:05 en_train
-rw-rw-r-- 1 chinh chinh  1249342 Thg 2  25 17:05 en_valid
-rw-rw-r-- 1 chinh chinh  9044100 Thg 2  25 17:05 en_vi_iwslt.tar.gz
-rw-rw-r-- 1 chinh chinh  1624771 Thg 2  25 17:05 vi_test
-rw-rw-r-- 1 chinh chinh 12896000 Thg 2  25 17:05 vi_train
-rw-rw-r-- 1 chinh chinh  1609835 Thg 2  25 17:05 vi_valid


In [7]:
start = time.time()
import sent2vec
model = sent2vec.Sent2vecModel()
model.load_model('../../../wiki_unigrams.bin')

print('=== Ending, Total time (second): ', time.time() - start)

=== Ending, Total time (second):  2.1098673343658447


In [8]:
srcfile = "vi_train"
tarfile = "en_train"
# lst_srcfile = open(srcfile, "r").readlines()
# lst_tarfile = open(tarfile, "r").readlines()
lst_srcfile = vi_train
lst_tarfile = en_train

vec_lst_srcfile = model.embed_sentences(lst_srcfile)

Test Cosin similarity

In [9]:
import faiss

start = time.time()
vec_lst_srcfile = model.embed_sentences(lst_srcfile)
vec_lst_srcfile2 = model.embed_sentences(lst_srcfile)
x = vec_lst_srcfile
q = vec_lst_srcfile2
index = faiss.IndexFlatL2(600)
# index = faiss.index_factory(3, "Flat", faiss.METRIC_INNER_PRODUCT)
index.ntotal
faiss.normalize_L2(x)
index.add(x)
faiss.normalize_L2(q)
k = 6
distance, index = index.search(q, k)

print('=== Ending, Total time (second): ', time.time() - start)
print(len(distance))
print(len(index))

print(index)
print(distance)

=== Ending, Total time (second):  30.133038997650146
96764
96764
[[    0 24974 16540 29928 78452 67635]
 [    1 92468 10524 65096 43509 67425]
 [    2  6667 44281 72560  9804 60254]
 ...
 [96761 47352 60425 69693 13151  6910]
 [96762 82612 15782 61073 38233  2934]
 [   47    96   183   271    46    28]]
[[0.0000000e+00 3.0174375e-01 3.0174375e-01 3.0174375e-01 3.0573297e-01
  3.0772603e-01]
 [4.7683716e-07 1.7269063e-01 1.7269063e-01 1.7269063e-01 1.9397974e-01
  1.9412506e-01]
 [0.0000000e+00 1.7876124e-01 3.0202234e-01 3.0584693e-01 3.1188440e-01
  3.2952094e-01]
 ...
 [0.0000000e+00 7.7144980e-02 8.1669092e-02 8.9680672e-02 9.1209531e-02
  9.7134233e-02]
 [0.0000000e+00 9.2331767e-02 1.8194175e-01 1.9165027e-01 1.9634044e-01
  2.1067023e-01]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00]]


In [10]:
i = 6
j = k-1
print(index[i][j])
print("==================================")
print(distance[i][j])
a = vec_lst_srcfile[i]
# print(a)
b = vec_lst_srcfile[index[i][j]]
# TEST
dist_squared = np.sum(np.square(a - b))
print(dist_squared)

887
0.1174556
0.11745526


In [11]:
start = time.time()
saperate = " || "
threshold = 0.8
src_label = "S"
emb_label = "E"

out_file_write = open("vi_train_EM", 'w')
out_file_write_score = open("vi_train_EM_score", 'w')
out_file_write_factor = open("vi_train_EM_factor", 'w')

for i in range(len(lst_srcfile)):
    score = distance[i][1]
    content = ""
    content_label_arr = []
    if score >= threshold:
        best_simi_index = index[i][1]
        content = lst_srcfile[i].strip() + saperate + lst_tarfile[best_simi_index].strip()
        
        content_label = [src_label] * len(lst_srcfile[i].split())
        content_label.append(emb_label)
        content_label = content_label + [emb_label] * len(lst_tarfile[best_simi_index].split())
        
        content_label_arr = " ".join(content_label)
        
        
   
    else:
        content = lst_srcfile[i].strip()
        content_label = [src_label] * len(lst_srcfile[i].split())
        content_label_arr = " ".join(content_label)
        
        
    if i<10:
        print(content)
        print(len(content.split()))  
        print(len(content_label))
        print("======================")
        

    out_file_write.writelines(content+ "\n")    
    out_file_write_factor.writelines(content_label_arr + "\n")
    out_file_write_score.writelines(str(score) + "\n")
        

out_file_write.close()

done = time.time()
elapsed = done - start
print('=== Ending, Total time (second): ', elapsed)

Cứ như thể là chàng ca sĩ đang ngồi ngay cạnh bạn , thì thầm điều gì đó vào tai bạn .
22
22
Chúng sẽ nhanh chóng hỏng nếu không được bảo quản . Và nếu không có nguồn điện lưới ổn định , vắc xin hỏng dẫn đến trẻ em tử vong .
31
31
Bây giờ cho phép tôi hỏi các bạn câu hỏi này : Tuần trước tại TED , bao nhiêu trong số các bạn khi thấy tính dễ bị tổn thương dâng cao ở đây , nghĩ rằng nó chính là sự can đảm ?
44
44
Tài liệu lưu trữ của tôi gồm : 175 bộ film , âm bản 16mm tất cả những cuốn sách của tôi , của bố tôi , || So the team that was the poorest had gotten even poorer .
40
40
Vài tuần trước khi chúng tôi bầu chọn , Đại Học Lake Superior State công bố danh sách những từ bị truất của năm
24
24
Chính bạn và tôi là những người tạo nên số phận của chúng ta Chính chúng ta mở ra hay đóng lại cánh cửa trên con đường trước mặt hay con đường sau lưng . &quot;
36
36
Cuộc thí nghiệm này diễn ra vào khoảng vài năm trước , xê dịch với lạm phát thì đây là một khoản tiền lớn so với vài phút làm việc 